In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
#Pull data from Part 1(WeatherPy)
weather_data = "../WeatherPy/output_data/my_weather_df.csv"
weather_df = pd.read_csv(weather_data)
weather_df.dropna(inplace= True)
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Castro,-24.7911,-50.0119,84.99,31,0,8.50,BR,1600020000
1,1,Ondjiva,-17.0667,15.7333,83.48,14,3,6.38,AO,1600020000
2,2,Bosaso,11.2842,49.1816,90.07,60,3,3.40,SO,1600020000
3,3,Hermanus,-34.4187,19.2345,56.23,73,64,29.24,ZA,1600020000
4,4,Olinda,-8.0089,-34.8553,82.04,60,37,14.52,BR,1600020000


In [3]:
#Show how many cities you"re working with 
city_count = weather_df["City"].count()
print(f"There are {city_count} cities in the dataframe.")

There are 553 cities in the dataframe.


In [4]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
#Store the Latitude and longitude in variables 
locations = weather_df[["Lat", "Lng"]].astype(float)

#Store Humidity in a variable as well 
humid = weather_df["Humidity"].astype(float)

In [13]:
#Create the heat layer map 
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights = humid, dissipating = False,
                                 max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#Narrow down cities to fit criteria of the temperature being between 70 and 80 degrees
#Wind speed less than 10mph, and zero cloudiness

#First find the cities between 70 and 80 degrees
less80 = weather_df.loc[weather_df["Max Temp"]<80,:]
great70 = less80.loc[weather_df["Max Temp"]>70,:]
#great70.count()
#great70.head()

#Now narrow the data more to find cities with wind speed less than 10mph
wind10 = great70.loc[great70["Wind Speed"]<10,:]
#wind10.count()
#wind10.head()

#Now reduce the list even further by finding the cities with zero cloudiness
cloud0 = wind10.loc[wind10["Cloudiness"]==0,:]
#cloud0.count()
cloud0

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
136,136,Fortuna,40.5982,-124.1573,70.50,72,0,6.73,US,1600020000
158,158,Angoche,-16.2325,39.9086,74.68,79,0,8.41,MZ,1600020000
313,313,Caluquembe,-13.7833,14.6833,75.52,21,0,3.60,AO,1600020000
337,337,Erzin,36.9559,36.2000,78.80,79,0,3.15,TR,1600020000
410,410,Kiryat Gat,31.6100,34.7642,79.32,58,0,9.60,IL,1600020000
482,482,Quetta,30.1990,67.0097,75.22,20,0,2.08,PK,1600020000
509,509,Semme,46.1963,1.5007,78.89,33,0,1.59,FR,1600020000
519,519,Jāwā,31.8525,35.9393,79.12,42,0,6.91,JO,1600020000
547,547,Alyangula,-13.8483,136.4192,72.34,86,0,7.29,AU,1600020000


In [8]:
#Create the hotel dataframe 
hotel_df = pd.DataFrame()
hotel_df["City"] = cloud0["City"]
hotel_df["Country"] = cloud0["Country"]
hotel_df["Lat"] = cloud0["Lat"]
hotel_df["Lng"] = cloud0["Lng"]
hotel_df["Hotel Name"]= ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
136,Fortuna,US,40.5982,-124.1573,
158,Angoche,MZ,-16.2325,39.9086,
313,Caluquembe,AO,-13.7833,14.6833,
337,Erzin,TR,36.9559,36.2000,
410,Kiryat Gat,IL,31.6100,34.7642,
482,Quetta,PK,30.1990,67.0097,
509,Semme,FR,46.1963,1.5007,
519,Jāwā,JO,31.8525,35.9393,
547,Alyangula,AU,-13.8483,136.4192,


In [9]:
#Find the closest hotel to each of the cities that meet the critiera

for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']
   
    #As the for loop iterates through the DF, print the name of each hotel as it finds the data
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    #Get results 
    results = response["results"]
    
    try:
        hotel = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotel}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotel
        
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("--------------------")
    
print("All hotels found in cities provided")  
        

Retrieving Results for Index 136: Fortuna.
Closest hotel to Fortuna at 40.5982 , -124.1573 is RWS Services.
--------------------
Retrieving Results for Index 158: Angoche.
Closest hotel to Angoche at -16.2325 , 39.9086 is Boy's guest house.
--------------------
Retrieving Results for Index 313: Caluquembe.
Missing field/result... skipping.
--------------------
Retrieving Results for Index 337: Erzin.
Closest hotel to Erzin at 36.9559 , 36.2 is Erzin Devlet Hastanesi.
--------------------
Retrieving Results for Index 410: Kiryat Gat.
Closest hotel to Kiryat Gat at 31.61 , 34.7642 is Shupersol Kiryat Gat.
--------------------
Retrieving Results for Index 482: Quetta.
Closest hotel to Quetta at 30.199 , 67.0097 is Zarmina Clinic.
--------------------
Retrieving Results for Index 509: Semme.
Closest hotel to Semme at 46.1963 , 1.5007 is Town Hall.
--------------------
Retrieving Results for Index 519: Jāwā.
Closest hotel to Jāwā at 31.8525 , 35.9393 is Jafra Plastic Industries Co.
--------

In [10]:
#Check if hotels were placed into Data Frame
hotel_df

,City,Country,Lat,Lng,Hotel Name
136,Fortuna,US,40.5982,-124.1573,RWS Services
158,Angoche,MZ,-16.2325,39.9086,Boy's guest house
313,Caluquembe,AO,-13.7833,14.6833,
337,Erzin,TR,36.9559,36.2000,Erzin Devlet Hastanesi
410,Kiryat Gat,IL,31.6100,34.7642,Shupersol Kiryat Gat
482,Quetta,PK,30.1990,67.0097,Zarmina Clinic
509,Semme,FR,46.1963,1.5007,Town Hall
519,Jāwā,JO,31.8525,35.9393,Jafra Plastic Industries Co
547,Alyangula,AU,-13.8483,136.4192,Alyangula Health Clinic


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [14]:
#Set the layer for the hotel markers
marker_layer = gmaps.marker_layer(hotel_locations, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))